In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.
import kagglehub
uwrfkaggler_ravdess_emotional_speech_audio_path = kagglehub.dataset_download('uwrfkaggler/ravdess-emotional-speech-audio')

print('Data source import complete.')


Using Colab cache for faster access to the 'ravdess-emotional-speech-audio' dataset.
Data source import complete.


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/ravdess-emotional-speech-audio/Actor_02/03-01-08-01-01-01-02.wav
/kaggle/input/ravdess-emotional-speech-audio/Actor_02/03-01-01-01-01-01-02.wav
/kaggle/input/ravdess-emotional-speech-audio/Actor_02/03-01-07-02-01-02-02.wav
/kaggle/input/ravdess-emotional-speech-audio/Actor_02/03-01-07-01-01-02-02.wav
/kaggle/input/ravdess-emotional-speech-audio/Actor_02/03-01-01-01-02-01-02.wav
/kaggle/input/ravdess-emotional-speech-audio/Actor_02/03-01-06-02-02-01-02.wav
/kaggle/input/ravdess-emotional-speech-audio/Actor_02/03-01-04-01-02-01-02.wav
/kaggle/input/ravdess-emotional-speech-audio/Actor_02/03-01-01-01-01-02-02.wav
/kaggle/input/ravdess-emotional-speech-audio/Actor_02/03-01-02-01-01-02-02.wav
/kaggle/input/ravdess-emotional-speech-audio/Actor_02/03-01-03-01-01-01-02.wav
/kaggle/input/ravdess-emotional-speech-audio/Actor_02/03-01-07-01-02-02-02.wav
/kaggle/input/ravdess-emotional-speech-audio/Actor_02/03-01-05-01-01-02-02.wav
/kaggle/input/ravdess-emotional-speech-audio/Actor_0

In [ ]:
!pip install soundfile

In [ ]:
from sklearn.model_selection import train_test_split
from tqdm import tqdm  # hiển thị tiến độ
import soundfile as sf

In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("uwrfkaggler/ravdess-emotional-speech-audio")

print("Path to dataset files:", path)

Using Colab cache for faster access to the 'ravdess-emotional-speech-audio' dataset.
Path to dataset files: /kaggle/input/ravdess-emotional-speech-audio


In [ ]:
import pandas as pd

# --- Set the correct path for the Kaggle environment ---
RAVDESS_PATH = '/kaggle/input/ravdess-emotional-speech-audio/'

# --- Define a mapping for the emotions based on the dataset description ---
emotion_map = {
    "01": "neutral",
    "02": "calm",
    "03": "happy",
    "04": "sad",
    "05": "angry",
    "06": "fearful",
    "07": "disgust",
    "08": "surprised"
}

def parse_ravdess_filename(filename):
    """Parses the RAVDESS filename to extract metadata."""
    try:
        parts = filename.split('.')[0].split('-')
        return {
            'modality': int(parts[0]),
            'vocal_channel': int(parts[1]),
            'emotion': emotion_map[parts[2]],
            'intensity': int(parts[3]),
            'statement': int(parts[4]),
            'repetition': int(parts[5]),
            'actor': int(parts[6]),
            'gender': 'female' if int(parts[6]) % 2 == 0 else 'male'
        }
    except:
        return None

# --- Loop through all files and build a list of metadata dictionaries ---
file_metadata = []
# We need to walk through the actor subdirectories inside the main path
for actor_folder in os.listdir(RAVDESS_PATH):
    actor_path = os.path.join(RAVDESS_PATH, actor_folder)
    if os.path.isdir(actor_path):
        for filename in os.listdir(actor_path):
            if filename.endswith('.wav'):
                metadata = parse_ravdess_filename(filename)
                if metadata:
                    metadata['file_path'] = os.path.join(actor_path, filename)
                    file_metadata.append(metadata)

# --- Create a pandas DataFrame ---
if not file_metadata:
    print(f"Error: No audio files were found or parsed in the directory: '{RAVDESS_PATH}'")
else:
    df = pd.DataFrame(file_metadata)
    # Filter for only the speech audio files (vocal_channel = 01)
    df = df[df['vocal_channel'] == 1].reset_index(drop=True)

    print("## Metadata DataFrame Created Successfully ##")
    print(f"Total speech files found: {len(df)}")
    print(df.head())

## Metadata DataFrame Created Successfully ##
Total speech files found: 1440
   modality  vocal_channel    emotion  intensity  statement  repetition  \
0         3              1  surprised          1          1           1   
1         3              1    neutral          1          1           1   
2         3              1    disgust          2          1           2   
3         3              1    disgust          1          1           2   
4         3              1    neutral          1          2           1   

   actor  gender                                          file_path  
0      2  female  /kaggle/input/ravdess-emotional-speech-audio/A...  
1      2  female  /kaggle/input/ravdess-emotional-speech-audio/A...  
2      2  female  /kaggle/input/ravdess-emotional-speech-audio/A...  
3      2  female  /kaggle/input/ravdess-emotional-speech-audio/A...  
4      2  female  /kaggle/input/ravdess-emotional-speech-audio/A...  


In [ ]:
df[:10]

,modality,vocal_channel,emotion,intensity,statement,repetition,actor,gender,file_path
0,3,1,surprised,1,1,1,2,female,/kaggle/input/ravdess-emotional-speech-audio/A...
1,3,1,neutral,1,1,1,2,female,/kaggle/input/ravdess-emotional-speech-audio/A...
2,3,1,disgust,2,1,2,2,female,/kaggle/input/ravdess-emotional-speech-audio/A...
3,3,1,disgust,1,1,2,2,female,/kaggle/input/ravdess-emotional-speech-audio/A...
4,3,1,neutral,1,2,1,2,female,/kaggle/input/ravdess-emotional-speech-audio/A...
5,3,1,fearful,2,2,1,2,female,/kaggle/input/ravdess-emotional-speech-audio/A...
6,3,1,sad,1,2,1,2,female,/kaggle/input/ravdess-emotional-speech-audio/A...
7,3,1,neutral,1,1,2,2,female,/kaggle/input/ravdess-emotional-speech-audio/A...
8,3,1,calm,1,1,2,2,female,/kaggle/input/ravdess-emotional-speech-audio/A...
9,3,1,happy,1,1,1,2,female,/kaggle/input/ravdess-emotional-speech-audio/A...


In [ ]:
import librosa
import numpy as np
import random
from tqdm import tqdm
from sklearn.model_selection import train_test_split


# ==========================
# 1️⃣ HÀM TĂNG DỮ LIỆU (DATA AUGMENTATION)
# ==========================
def augment_audio(y, sr):
    """
    Sinh thêm 3 biến thể của tín hiệu âm thanh gốc:
    - Noise Injection
    - Pitch Shift
    - Time Stretch
    """
    augmented = []

    # 1️⃣ Noise Injection (thêm tiếng ồn)
    noise = np.random.randn(len(y))
    y_noise = y + 0.005 * noise
    augmented.append(y_noise)

    # 2️⃣ Pitch Shift (thay đổi cao độ)
    n_steps = random.choice([-2, -1, 1, 2])
    y_pitch = librosa.effects.pitch_shift(y=y, sr=sr, n_steps=2)
    augmented.append(y_pitch)

    # 3️⃣ Time Stretch (thay đổi tốc độ nói)
    rate = random.choice([0.8, 1.2])
    y_stretch = librosa.effects.time_stretch(y, rate=rate)
    augmented.append(y_stretch)

    return augmented


# ==========================
# 2️⃣ HÀM TRÍCH XUẤT ĐẶC TRƯNG (FEATURE EXTRACTION)
# ==========================
def extract_feature_from_raw(y, sr):
    """
    Trích xuất đặc trưng trực tiếp từ dữ liệu audio (numpy array)
    """
    try:
        result = np.array([])

        # --- MFCC ---
        mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=40)
        mfccs = np.mean(mfccs, axis=1)
        result = np.hstack((result, mfccs))

        # --- CHROMA ---
        stft = np.abs(librosa.stft(y))
        chroma = librosa.feature.chroma_stft(S=stft, sr=sr)
        chroma = np.mean(chroma, axis=1)
        result = np.hstack((result, chroma))

        # --- MEL SPECTROGRAM ---
        mel = librosa.feature.melspectrogram(y=y, sr=sr)
        mel = np.mean(mel, axis=1)
        result = np.hstack((result, mel))

        return result

    except Exception as e:
        print(f" Lỗi khi trích xuất đặc trưng: {e}")
        return None


# ==========================
# 3️⃣ LOAD DATAFRAME VÀ TĂNG DỮ LIỆU
# ==========================
def load_data_with_augmentation(df, test_size=0.2):
    """
    Chia train/test TRƯỚC, sau đó chỉ tăng cường (augment) dữ liệu cho tập train.
    """

    # --- 1️⃣ Chia dữ liệu gốc trước ---
    train_df, test_df = train_test_split(df, test_size=test_size, random_state=42, stratify=df['emotion'])

    X_train, y_train, X_test, y_test = [], [], [], []

    print("🔹 Đang trích xuất đặc trưng cho tập TRAIN (có augment)...\n")
    for idx, row in tqdm(train_df.iterrows(), total=len(train_df)):
        file_path = row['file_path']
        emotion = row['emotion']

        try:
            # Load âm thanh gốc
            y_raw, sr = librosa.load(file_path, sr=None, mono=True)

            # 🔹 Trích xuất đặc trưng gốc
            features = extract_feature_from_raw(y_raw, sr)
            if features is not None:
                X_train.append(features)
                y_train.append(emotion)

            # 🔹 Augment chỉ trong tập train
            augmented_data = augment_audio(y_raw, sr)
            for y_aug in augmented_data:
                features_aug = extract_feature_from_raw(y_aug, sr)
                if features_aug is not None:
                    X_train.append(features_aug)
                    y_train.append(emotion)

        except Exception as e:
            print(f"⚠️ Lỗi khi xử lý file train {file_path}: {e}")
            continue

    print("\n🔹 Đang trích xuất đặc trưng cho tập TEST (không augment)...\n")
    for idx, row in tqdm(test_df.iterrows(), total=len(test_df)):
        file_path = row['file_path']
        emotion = row['emotion']

        try:
            y_raw, sr = librosa.load(file_path, sr=None, mono=True)
            features = extract_feature_from_raw(y_raw, sr)
            if features is not None:
                X_test.append(features)
                y_test.append(emotion)
        except Exception as e:
            print(f"⚠️ Lỗi khi xử lý file test {file_path}: {e}")
            continue

    # --- 2️⃣ Chuyển sang numpy ---
    X_train = np.vstack(X_train)
    y_train = np.array(y_train)
    X_test = np.vstack(X_test)
    y_test = np.array(y_test)

    print("\n✅ Hoàn tất trích xuất + tăng cường dữ liệu!")
    print(f"   Tập train (sau augment): {len(X_train)} mẫu")
    print(f"   Tập test: {len(X_test)} mẫu")
    print(f"   Kích thước vector đặc trưng: {X_train.shape[1]}")

    return X_train, X_test, y_train, y_test


# ==========================
# 4️⃣ CHẠY THỬ
# ==========================
X_train, X_test, y_train, y_test = load_data_with_augmentation(df)

🔹 Đang trích xuất đặc trưng cho tập TRAIN (có augment)...



100%|██████████| 1152/1152 [11:04<00:00,  1.73it/s]



🔹 Đang trích xuất đặc trưng cho tập TEST (không augment)...



100%|██████████| 288/288 [00:25<00:00, 11.11it/s]


✅ Hoàn tất trích xuất + tăng cường dữ liệu!
   Tập train (sau augment): 4608 mẫu
   Tập test: 288 mẫu
   Kích thước vector đặc trưng: 180


In [ ]:
files = sorted(glob.glob("/kaggle/input/ravdess-emotional-speech-audio/Actor_*/*.wav"))
print(len(files))
print(files[:5])  # xem 5 file đầu tiên

1440
['/kaggle/input/ravdess-emotional-speech-audio/Actor_01/03-01-01-01-01-01-01.wav', '/kaggle/input/ravdess-emotional-speech-audio/Actor_01/03-01-01-01-01-02-01.wav', '/kaggle/input/ravdess-emotional-speech-audio/Actor_01/03-01-01-01-02-01-01.wav', '/kaggle/input/ravdess-emotional-speech-audio/Actor_01/03-01-01-01-02-02-01.wav', '/kaggle/input/ravdess-emotional-speech-audio/Actor_01/03-01-02-01-01-01-01.wav']


In [ ]:
y_test[:10]

array(['sad', 'calm', 'happy', 'angry', 'neutral', 'angry', 'angry',
       'fearful', 'disgust', 'angry'], dtype='<U9')

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix


dtree_model = DecisionTreeClassifier(max_depth = 15).fit(X_train, y_train)
dtree_predictions = dtree_model.predict(X_test)

print(accuracy_score(y_true=y_test,y_pred=dtree_predictions))
print(classification_report(y_test,dtree_predictions))
# creating a confusion matrix
print(confusion_matrix(y_test, dtree_predictions) )

0.375
              precision    recall  f1-score   support

       angry       0.43      0.53      0.48        38
        calm       0.60      0.39      0.48        38
     disgust       0.46      0.32      0.38        38
     fearful       0.44      0.38      0.41        39
       happy       0.26      0.36      0.30        39
     neutral       0.35      0.32      0.33        19
         sad       0.33      0.29      0.31        38
   surprised       0.28      0.38      0.32        39

    accuracy                           0.38       288
   macro avg       0.40      0.37      0.38       288
weighted avg       0.40      0.38      0.38       288

[[20  0  2  3  4  2  2  5]
 [ 1 15  3  1  2  6  8  2]
 [ 4  2 12  4  5  0  5  6]
 [ 5  1  1 15  6  1  2  8]
 [ 6  1  2  6 14  1  2  7]
 [ 1  2  2  1  2  6  2  3]
 [ 2  2  1  2 11  1 11  8]
 [ 7  2  3  2  9  0  1 15]]


In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix


dtree_model = DecisionTreeClassifier(max_depth = 20).fit(X_train, y_train)
dtree_predictions = dtree_model.predict(X_test)

print(accuracy_score(y_true=y_test,y_pred=dtree_predictions))
print(classification_report(y_test,dtree_predictions))
# creating a confusion matrix
print(confusion_matrix(y_test, dtree_predictions) )

0.3680555555555556
              precision    recall  f1-score   support

       angry       0.48      0.53      0.50        38
        calm       0.57      0.45      0.50        38
     disgust       0.43      0.34      0.38        38
     fearful       0.42      0.41      0.42        39
       happy       0.30      0.41      0.35        39
     neutral       0.30      0.32      0.31        19
         sad       0.29      0.21      0.24        38
   surprised       0.21      0.26      0.23        39

    accuracy                           0.37       288
   macro avg       0.37      0.36      0.37       288
weighted avg       0.38      0.37      0.37       288

[[20  0  3  3  5  1  1  5]
 [ 0 17  2  2  3  5  7  2]
 [ 4  2 13  5  4  1  4  5]
 [ 4  3  2 16  5  0  4  5]
 [ 5  1  3  4 16  4  0  6]
 [ 0  4  2  1  2  6  2  2]
 [ 2  1  1  4  9  1  8 12]
 [ 7  2  4  3  9  2  2 10]]


In [ ]:
from sklearn.svm import SVC
svm_model_linear = SVC(kernel = 'linear', C = 1).fit(X_train, y_train)
svm_predictions = svm_model_linear.predict(X_test)

print(accuracy_score(y_true=y_test,y_pred=svm_predictions))
print(classification_report(y_test,svm_predictions))
# creating a confusion matrix
print(confusion_matrix(y_test, svm_predictions) )

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rf_model = RandomForestClassifier(
    n_estimators=300,
    max_depth=12,
    random_state=42,
    class_weight='balanced'
)
rf_model.fit(X_train, y_train)
y_pred_rf = rf_model.predict(X_test)

print("Accuracy:", accuracy_score(y_test, y_pred_rf))
print(classification_report(y_test, y_pred_rf))

Accuracy: 0.5694444444444444
              precision    recall  f1-score   support

       angry       0.70      0.55      0.62        38
        calm       0.62      0.84      0.71        38
     disgust       0.50      0.74      0.60        38
     fearful       0.64      0.54      0.58        39
       happy       0.56      0.56      0.56        39
     neutral       0.50      0.42      0.46        19
         sad       0.67      0.16      0.26        38
   surprised       0.49      0.67      0.57        39

    accuracy                           0.57       288
   macro avg       0.58      0.56      0.54       288
weighted avg       0.59      0.57      0.55       288



In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

knn = KNeighborsClassifier(n_neighbors=7)
knn.fit(X_train_scaled, y_train)
y_pred_knn = knn.predict(X_test_scaled)

print("Accuracy:", accuracy_score(y_test, y_pred_knn))
print(classification_report(y_test, y_pred_knn))

Accuracy: 0.6041666666666666
              precision    recall  f1-score   support

       angry       0.55      0.68      0.61        38
        calm       0.60      0.76      0.67        38
     disgust       0.81      0.68      0.74        38
     fearful       0.56      0.59      0.57        39
       happy       0.61      0.56      0.59        39
     neutral       0.50      0.42      0.46        19
         sad       0.62      0.34      0.44        38
   surprised       0.57      0.69      0.63        39

    accuracy                           0.60       288
   macro avg       0.60      0.59      0.59       288
weighted avg       0.61      0.60      0.60       288



In [ ]:

from sklearn.ensemble import RandomForestClassifier

 # create regressor object
classifier = RandomForestClassifier(n_estimators = 300,random_state = 0)

# fit the regressor with x and y data
classifier.fit(X_train, y_train)

c_p = classifier.predict(X_test)


print(accuracy_score(y_true=y_test,y_pred=c_p))
print(classification_report(y_test,c_p))
# creating a confusion matrix
print(confusion_matrix(y_test,c_p) )

0.5376044568245125
              precision    recall  f1-score   support

       angry       0.82      0.56      0.67        48
        calm       0.45      0.83      0.58        47
     disgust       0.44      0.60      0.51        45
     fearful       0.67      0.49      0.56        45
       happy       0.38      0.55      0.45        38
     neutral       0.57      0.32      0.41        25
         sad       0.65      0.45      0.53        53
   surprised       0.66      0.43      0.52        58

    accuracy                           0.54       359
   macro avg       0.58      0.53      0.53       359
weighted avg       0.59      0.54      0.54       359

[[27  0  9  2  4  0  0  6]
 [ 0 39  1  1  2  3  1  0]
 [ 1  6 27  0  9  0  0  2]
 [ 2  5  5 22  5  0  5  1]
 [ 2  4  6  1 21  1  2  1]
 [ 0 15  0  1  1  8  0  0]
 [ 0 13  4  5  4  0 24  3]
 [ 1  5  9  1 10  2  5 25]]


In [ ]:

from sklearn.neural_network import MLPClassifier

from sklearn.metrics import accuracy_score

import os
import pickle


In [ ]:

mn_params = {
    'alpha': 0.01,
    'batch_size': 256,
    'epsilon': 1e-08,
    'hidden_layer_sizes': (300,),
    'learning_rate': 'adaptive',
    'max_iter': 500,
}
# initialize Multi Layer Perceptron classifier
# with best parameters ( so far )
m2 = MLPClassifier(**mn_params)

# train the model
print("[*] Training the model...")
m2.fit(X_train, y_train)

# predict 25% of data to measure how good we are
y_p2 = m2.predict(X_test)

# calculate the accuracy
accuracy = accuracy_score(y_true=y_test, y_pred=y_p2)

print("Accuracy: {:.2f}%".format(accuracy*100))

[*] Training the model...
Accuracy: 55.90%


In [ ]:
from sklearn.preprocessing import LabelEncoder

# Khởi tạo encoder
le = LabelEncoder()

# Encode nhãn
y_train_enc = le.fit_transform(y_train)
y_test_enc = le.transform(y_test)

# Kiểm tra mapping
print("Mapping:", dict(zip(le.classes_, le.transform(le.classes_))))

Mapping: {np.str_('angry'): np.int64(0), np.str_('calm'): np.int64(1), np.str_('disgust'): np.int64(2), np.str_('fearful'): np.int64(3), np.str_('happy'): np.int64(4), np.str_('neutral'): np.int64(5), np.str_('sad'): np.int64(6), np.str_('surprised'): np.int64(7)}


In [ ]:
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

xgb_model = XGBClassifier(
    n_estimators=500,
    learning_rate=0.05,
    max_depth=8,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42
)

xgb_model.fit(X_train, y_train_enc)
y_pred_xgb = xgb_model.predict(X_test)

y_pred_labels = le.inverse_transform(y_pred_xgb)
y_test_labels = y_test  # y_test gốc là dạng string

print(" Accuracy:", accuracy_score(y_test_labels, y_pred_labels))
print(classification_report(y_test_labels, y_pred_labels))
print(confusion_matrix(y_test_labels, y_pred_labels))

 Accuracy: 0.7013888888888888
              precision    recall  f1-score   support

       angry       0.81      0.76      0.78        38
        calm       0.77      0.87      0.81        38
     disgust       0.81      0.76      0.78        38
     fearful       0.70      0.82      0.75        39
       happy       0.63      0.74      0.68        39
     neutral       0.71      0.53      0.61        19
         sad       0.62      0.42      0.50        38
   surprised       0.59      0.62      0.60        39

    accuracy                           0.70       288
   macro avg       0.70      0.69      0.69       288
weighted avg       0.70      0.70      0.70       288

[[29  0  3  0  4  0  0  2]
 [ 1 33  0  0  2  1  1  0]
 [ 0  1 29  1  1  0  0  6]
 [ 2  1  0 32  0  0  2  2]
 [ 2  1  1  4 29  0  0  2]
 [ 0  1  1  0  1 10  5  1]
 [ 0  6  1  7  3  1 16  4]
 [ 2  0  1  2  6  2  2 24]]


In [ ]:
from sklearn.svm import SVC
svm_model_linear = SVC(kernel = 'linear', C = 1).fit(X_train, y_train)
svm_predictions = svm_model_linear.predict(X_test)


print(accuracy_score(y_true=y_test,y_pred=svm_predictions))
print(classification_report(y_test,svm_predictions))
# creating a confusion matrix
print(confusion_matrix(y_test, svm_predictions) )


0.4791666666666667
              precision    recall  f1-score   support

       angry       0.52      0.61      0.56        38
        calm       0.56      0.76      0.64        38
     disgust       0.48      0.42      0.45        38
     fearful       0.49      0.54      0.51        39
       happy       0.49      0.51      0.50        39
     neutral       0.00      0.00      0.00        19
         sad       0.34      0.26      0.30        38
   surprised       0.46      0.49      0.47        39

    accuracy                           0.48       288
   macro avg       0.42      0.45      0.43       288
weighted avg       0.45      0.48      0.46       288

[[23  0  3  2  4  1  2  3]
 [ 0 29  4  2  1  0  2  0]
 [ 8  3 16  4  2  0  2  3]
 [ 4  1  2 21  3  0  4  4]
 [ 3  3  1  4 20  2  1  5]
 [ 1  7  3  1  1  0  3  3]
 [ 3  7  2  6  5  1 10  4]
 [ 2  2  2  3  5  1  5 19]]


In [ ]:
from sklearn.neural_network import MLPClassifier

mlp = MLPClassifier(hidden_layer_sizes=(256,128,64), activation='relu',
                    solver='adam', learning_rate_init=0.001, max_iter=500)
mlp.fit(X_train, y_train)
y_pred_mlp = mlp.predict(X_test)

print("Accuracy:", accuracy_score(y_test, y_pred_mlp))
print(classification_report(y_test, y_pred_mlp))

Accuracy: 0.4722222222222222
              precision    recall  f1-score   support

       angry       0.72      0.74      0.73        38
        calm       0.61      0.29      0.39        38
     disgust       0.24      0.84      0.37        38
     fearful       0.73      0.62      0.67        39
       happy       0.71      0.38      0.50        39
     neutral       0.00      0.00      0.00        19
         sad       0.80      0.21      0.33        38
   surprised       0.60      0.46      0.52        39

    accuracy                           0.47       288
   macro avg       0.55      0.44      0.44       288
weighted avg       0.59      0.47      0.47       288

